In [1]:
#建立連結
from py2neo import  Graph, Node, Relationship, NodeMatcher
graph = Graph(scheme="bolt", host="localhost", port=7687, secure = False) #auth改成自己的帳密

In [2]:
#建立Course Node
with open("./Course_noIntro.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        ClassName,ClassID,ClassData,ClassPre,ClassStop,ClassTeacher,ClassSchool=c.split(",")
        ClassName=ClassName.strip()
        node=Node("Courses",name=ClassName,ID=ClassID,data=ClassData,pre=ClassPre,stop=ClassStop,teacher=ClassTeacher,school=ClassSchool)
        graph.create(node)

In [3]:
#建立Professor Node
with open("./Professor.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        professorID, professorName, professorDepartment,professorSchool = c.split(",")
        node=Node("Professors",name=professorName,ID=professorID,Department=professorDepartment,School=professorSchool)
        graph.create(node)

In [4]:
#建立Student Node
with open("./OnlyStudent.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        StudentID, StudentName = c.split(",")
        StudentName = StudentName.rstrip()
        node=Node("Students",name=StudentName,ID=StudentID, password="000000")
        graph.create(node)

In [7]:
#建立Courses Relation
with open("./Course_noIntro.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        ClassName,ClassID,ClassData,ClassPre,ClassStop,ClassTeacher,ClassSchool=c.split(",")
        #先修
        PreClass=[]
        eachPreClass=ClassPre.split("、")
        for i in eachPreClass:
            if(len(i)<3):
                break
            PreClass.append(i)
        #擋修
        StopClass=[]
        eachClass=ClassStop.split("、")
        for i in eachClass:
            if(len(i)<3):
                break
            StopClass.append(i)
        
        matcher = NodeMatcher(graph)
        #處理先修
        for i in PreClass:     
            existing_u1 = matcher.match("Courses", name=ClassName).first()
            existing_u2 = matcher.match("Courses", name=i).first()
            existing_u1_pre_u2 = Relationship(existing_u1, '先修', existing_u2)
            graph.create(existing_u1_pre_u2)
        #處理擋修
        for i in StopClass:
            existing_u1 = matcher.match("Courses", name=ClassName).first()
            existing_u2 = matcher.match("Courses", name=i).first()
            existing_u1_stop_u2 = Relationship(existing_u1, '擋修', existing_u2)
            graph.create(existing_u1_stop_u2)
        #處理教授
        existing_u1 = matcher.match("Professors", name=ClassTeacher).first()
        existing_u2 = matcher.match("Courses", teacher=ClassTeacher)
        result = list(existing_u2)
        for i in range(len(result)):
            existing_u1_teach_u2 = Relationship(existing_u1, '教', result[i])
            graph.create(existing_u1_teach_u2)

In [8]:
#建立Student Relation
with open("./Student.csv", "r",encoding='big5') as f:
    content = f.readlines()
    for index, c in enumerate(content):
        if(index==0):
            continue
        StudentID, StudentName, ClassName, ClassTime, StudentSchool = c.split(",")
        StudentName=StudentName.rstrip()
        ClassName=ClassName.rstrip()
        existing_u1 = matcher.match("Students", name = StudentName).first()
        existing_u2 = matcher.match("Courses", name = ClassName)  
        existing_u1_teach_u2 = Relationship(existing_u1, '修', existing_u2, taken_time = ClassTime)
        graph.create(existing_u1_teach_u2)